In [21]:
# Import Libraries
import pandas as pd
import datetime
import math
import numpy as np

In [22]:
# Read file and print the number of records and data types
csv_name = "./2019/station2019.csv"
station_df = pd.read_csv(csv_name, low_memory=False) 
# print("Stations for 2019 : " + "\n" + str(station_df.count()))
station_df = station_df[station_df['year'].notna()]
uniqueid_df = station_df.drop_duplicates(subset=['stationid'], keep='first')
uniqueid_df

,year,month,stationid,startstatname,start station latitude,start station longitude,starttrips,endstatname,end station latitude,end station longitude,endtrips
0,2019.0,Jan,72.0,W 52 St & 11 Ave,40.767272,-73.993929,1697.0,W 52 St & 11 Ave,40.767272,-73.993929,1662.0
1,2019.0,Jan,79.0,Franklin St & W Broadway,40.719116,-74.006667,1271.0,Franklin St & W Broadway,40.719116,-74.006667,1284.0
2,2019.0,Jan,82.0,St James Pl & Pearl St,40.711174,-74.000165,627.0,St James Pl & Pearl St,40.711174,-74.000165,633.0
3,2019.0,Jan,83.0,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,390.0,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,584.0
4,2019.0,Jan,119.0,Park Ave & St Edwards St,40.696089,-73.978034,232.0,Park Ave & St Edwards St,40.696089,-73.978034,308.0
...,...,...,...,...,...,...,...,...,...,...,...
9925,2019.0,Dec,3902.0,Fairview Ave & Linden St,40.706940,-73.906320,8.0,Fairview Ave & Linden St,40.706940,-73.906320,11.0
9927,2019.0,Dec,3907.0,Central Ave & Melrose St,40.701120,-73.930390,49.0,Central Ave & Melrose St,40.701120,-73.930390,35.0
9929,2019.0,Dec,3909.0,Madison St & Woodward Ave,40.704080,-73.904080,6.0,Madison St & Woodward Ave,40.704080,-73.904080,6.0
9930,2019.0,Dec,3910.0,Greene Ave & Grandview Ave,40.709697,-73.907856,30.0,Greene Ave & Grandview Ave,40.709697,-73.907856,34.0


In [ ]:
df = uniqueid_df[['stationid','end station latitude', 'end station longitude']]
df.rename(columns={"end station latitude":"lat", "end station longitude":"lon"}, inplace=True)
df = df[df['lat'].notna()]

from tqdm import tqdm
tqdm.pandas()

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="NY_City")

from geopy.extra.rate_limiter import RateLimiter
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=0.05)

df['Location'] = df.progress_apply(lambda row: reverse((row['lat'], row['lon'])),axis=1)


def parse_zipcode(location):
     if location and location.raw.get('address') and location.raw['address'].get('postcode'):
        return location.raw['address']['postcode']
     else:
        return None
df['Zipcode'] = df['Location'].apply(parse_zipcode)

# df

/Users/user/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/Users/user/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
  4%|▍         | 38/926 [00:18<07:06,  2.08it/s]

In [17]:
df.to_csv('./2019/stations_zipcodes.csv', encoding='utf-8')
csv = "./2019/stations_zipcodes.csv"
df = pd.read_csv(csv, low_memory=False) 
df

,Unnamed: 0,stationid,lat,lon,Location,Zipcode
0,0,72.0,40.767272,-73.993929,"Citi Bike - W 52 St & 11 Ave, West 52nd Street...",10019
1,1,79.0,40.719116,-74.006667,"Citi Bike - Franklin St & W Broadway, Franklin...",10005
2,2,82.0,40.711174,-74.000165,"Citi Bike - St James Pl & Pearl St, Pearl Stre...",10038
3,3,83.0,40.683826,-73.976323,"Citi Bike - Atlantic Ave & Fort Greene Pl, Atl...",11208
4,4,119.0,40.696089,-73.978034,"Citi Bike - Park Ave & St Edwards St, Park Ave...",11251
...,...,...,...,...,...,...
921,9925,3902.0,40.706940,-73.906320,"618, Fairview Avenue, Ridgewood, Queens, Queen...",11385
922,9927,3907.0,40.701120,-73.930390,"67, Central Avenue, Bushwick, Brooklyn, Kings ...",11206
923,9929,3909.0,40.704080,-73.904080,"BMT Myrtle Avenue Line, Harrison Place, Brookl...",11237
924,9930,3910.0,40.709697,-73.907856,"Post Box, Greene Avenue, Bushwick, Brooklyn, K...",11237


In [20]:
stat_zipcode = pd.merge(station_df, df, how="left", on=["stationid"])
stat_zipcode.to_csv('./2019/stations2019zipcode.csv', encoding='utf-8')
stat_zipcode

,year,month,stationid,startstatname,start station latitude,start station longitude,starttrips,endstatname,end station latitude,end station longitude,endtrips,Unnamed: 0,lat,lon,Location,Zipcode
0,2019.0,Jan,72.0,W 52 St & 11 Ave,40.767272,-73.993929,1697.0,W 52 St & 11 Ave,40.767272,-73.993929,1662.0,0.0,40.767272,-73.993929,"Citi Bike - W 52 St & 11 Ave, West 52nd Street...",10019
1,2019.0,Jan,79.0,Franklin St & W Broadway,40.719116,-74.006667,1271.0,Franklin St & W Broadway,40.719116,-74.006667,1284.0,1.0,40.719116,-74.006667,"Citi Bike - Franklin St & W Broadway, Franklin...",10005
2,2019.0,Jan,82.0,St James Pl & Pearl St,40.711174,-74.000165,627.0,St James Pl & Pearl St,40.711174,-74.000165,633.0,2.0,40.711174,-74.000165,"Citi Bike - St James Pl & Pearl St, Pearl Stre...",10038
3,2019.0,Jan,83.0,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,390.0,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,584.0,3.0,40.683826,-73.976323,"Citi Bike - Atlantic Ave & Fort Greene Pl, Atl...",11208
4,2019.0,Jan,119.0,Park Ave & St Edwards St,40.696089,-73.978034,232.0,Park Ave & St Edwards St,40.696089,-73.978034,308.0,4.0,40.696089,-73.978034,"Citi Bike - Park Ave & St Edwards St, Park Ave...",11251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10805,NaN,NaN,3213.0,NaN,NaN,NaN,NaN,Van Vorst Park,40.718489,-74.047727,1.0,NaN,NaN,NaN,NaN,NaN
10806,NaN,NaN,3245.0,NaN,NaN,NaN,NaN,NYCBS DEPOT - DELANCEY,40.716444,-73.982331,7.0,411.0,40.716444,-73.982331,"234, Delancey Street, Lower East Side, Manhatt...",10002
10807,NaN,NaN,3267.0,NaN,NaN,NaN,NaN,Morris Canal,40.712419,-74.038526,2.0,NaN,NaN,NaN,NaN,NaN
10808,NaN,NaN,3270.0,NaN,NaN,NaN,NaN,Jersey & 6th St,40.725289,-74.045572,1.0,NaN,NaN,NaN,NaN,NaN
